In [1]:
# 오라클DB 접속
import oml
import oracledb
oml.core.methods.__embed__ = False

connection = oracledb.connect(user="vector", password="vector", dsn="localhost:1521/freepdb1")
cursor = connection.cursor()

# 버전 확인
dbversion = """select product,VERSION_FULL from product_component_version"""
result=cursor.execute(dbversion).fetchall()
print('DB version : ',result)

DB version :  [('Oracle Database 23ai Free', '23.7.0.25.01')]


In [3]:
# # RAG with llama3.2 on ollama
# 언어 모델 : llama3.2
# 쿼리 임베딩 모델 : jeffh/intfloat-multilingual-e5-large-instruct:f16
# 임베딩 결과 리턴 : JSON_OBJECT in oracle
# APIs parameter : ollama API by oracle for embedding
# 임베딩 유틸리티 : utl_to_embedding
#  프롬프트 템플릿 APIs : Lang-chain-ollama 라이브러리 

# 실행 순서 : 1. 사용자 질문 -> 2. 질문(문장) 벡터검색 결과 추출 -> 3. 벡터 검색 결과로 프로프트 생성 -> 4. LLM 답변 생성 -> 5. LLM 응답 

# 1. LLM에 사용자 질문(user_q) 입력 = "품질관리 수준 평가 절차를 정리해줘"

user_q = input("질문을 입력해주세요?(예, 품질관리 수준 평가 절차를 정리해줘) \n : ")

print(" 프롬프트에 사용할 내용에 대한 벡터검색을 실행중입니다.")
# 2. 질문(문장) 벡터검색 결과 추출
top_k = 5

vector_q = """select doc_id,embed_id as Chunked_id, embed_data as Chunked_data
from doc_vector 
order by vector_distance(embed_vector,vector_embedding(MULTILINGUAL_E5_SMALL using :user_q as data),COSINE)
fetch first :top_k rows only
          """
cursor.execute(vector_q, user_q = user_q,  top_k=top_k)
rows = cursor.fetchall()
#content = rows
#connection.close()

import pandas as pd

tbl = pd.DataFrame(rows)

colnm = cursor.description
col=[]

for i in colnm:
    col.append(i[0])

tbl.columns = col

tbl['CHUNKED_DATA'] = tbl['CHUNKED_DATA'].str.replace("\n","", regex=False)
#print()

pd.set_option('display.max_colwidth', None)

#display(tbl)
print("백터검색 결과")
display(tbl)

질문을 입력해주세요?(예, 품질관리 수준 평가 절차를 정리해줘) 
 :  전시 군사법원 운영절차 요약해주세요


 프롬프트에 사용할 내용에 대한 벡터검색을 실행중입니다.
백터검색 결과


,DOC_ID,CHUNKED_ID,CHUNKED_DATA
0,17,11,"1의2. 전시 군사법원의 운영2. 군의 형사정책3. 법령의 제정ㆍ개정 및 법규의 관리4. 법령 해석 및 법률 자문5. 소송ㆍ배상 및 행정심판6. 징계에 관한 업무7. 계약안 및 조약안 검토8. 장병의 인권보장9. 참모총장이 명하는 법무업무에 관한 사항의 처리제7조의2(군사경찰실장)① 군사경찰실장은 장성급 장교로 보한다.<개정 2017. 9. 5., 2020. 2. 4.>② 군사경찰실장은 다음 각 호의 사항에 관하여 참모총장을 보좌한다."
1,7,606,"[본조신설 2021. 9. 24.]제534조의15(전시의 서기 등)① 전시 군사법원서기는 각 군 참모총장이 소속 장교, 준사관, 부사관 및 군무원 중에서 임명한다. 다만, 국방부의 전시 군사법원서기는 국방부장관이 임명한다. ② 법정경위는 군무원, 부사관 또는 병 중에서 관할관이 임명한다.③ 통역인과 기사는 장교 또는 군무원 중에서 관할관이 임명한다.[본조신설 2021. 9. 24.]제534조의16(전시 군검찰부)"
2,7,60,제533조(노역장 유치의 집행) 제5편 전시ㆍ사변 시의 특례 제534조(특례규정) 제534조의2(전시 군사법원의 종류) 제534조의3(전시 군사법원의 설치) 제534조의4(전시 군사법원의 관할관) 제534조의5(전시 군사법원의 심판사항) 제534조의6(전시 관련사건 관할의 병합과 예외) 제534조의7(보통군사법원의 판결에 대한 관할관의 확인조치) 제534조의8(전시 군사법원의 구성) 제534조의9(전시 군판사의 임명 및 소속)
3,7,594,"① 전시 군사법원의 행정사무를 관장하는 관할관(이하 ""관할관""이라 한다)을 둔다. ② 고등군사법원의 관할관은 국방부장관으로 한다.③ 보통군사법원의 관할관은 그 설치되는 부대와 지역의 사령관, 장 또는 책임지휘관으로 한다. 다만, 국방부 보통군사법원의 관할관은 고등군사법원의 관할관이 겸임한다.④ 고등군사법원의 관할관은 국방부와 각 군 본부 보통군사법원의 행정사무를 지휘ㆍ감독하고, 각 군 본부 보통군사법원의 관할관은 예하부대 보통군사법원의 행정사무를 지휘ㆍ감독한다.[본조신설 2021. 9. 24.]제534조의5(전시 군사법원의 심판사항)① 보통군사법원은 다음 각 호의 사건을 제1심으로 심판한다."
4,7,593,"전시ㆍ사변 또는 이에 준하는 국가비상사태 시의 군사법원(이하 ""전시 군사법원""이라 한다)은 다음 각 호의 두 종류로 한다.1. 고등군사법원2. 보통군사법원[본조신설 2021. 9. 24.]제534조의3(전시 군사법원의 설치)① 고등군사법원은 국방부에 설치한다. ② 국방부장관은 제6조에도 불구하고 편제상 장성급 장교가 지휘하는 부대 또는 기관에 보통군사법원을 설치할 수 있다.[본조신설 2021. 9. 24.]제534조의4(전시 군사법원의 관할관)"


In [ ]:
# 3. 벡터 검색 결과로 프로프트 생성
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

#print(tbl['DOC_ID'])

#ContentId = int(tbl.iloc[0, 0])
ContentId = input("백터검색을 참고하여 프롬프트의 컨텐트에 사용할 문서번호를 입력해주세요? : ")
content_q = """ SELECT dbms_vector_chain.utl_to_text(dt.laws,json('{
                                      "plaintext": "true",
                                      "charset"  : "UTF8" 
           }')) as text 
from laws_doc dt 
where docid = :content_id"""
cursor.execute(content_q,content_id=ContentId)
row = cursor.fetchall()
outputs = row[0][0].read()
#print(outputs)

import re
content = re.sub("[\n]", "", outputs)
#print(content)

print(user_q + " 에 대한 답변을 LLAMA3.2가 실행중입니다. 잠시후 결과를 확인하세요")

user_question = user_q

template = """
content 정보를 참조하여 다음 질문에 답변해주세요.
    
    답변시 참고 사항:
    - 참조 문서: 만약 답변에 특정 문서의 정보를 사용했다면, 해당 문서의 번호와 청크 번호를 명시해주세요.  
    - 답변 어조 : 질문에 대해서 사실 기반으로 답변해주세요. 답변 작성시 친숙한 어조로 작성해주세요. 
    - 언어: 답변은 한국어로 작성하며, 명확하고 간결하게 요약하여 500자 이내로 제한합니다.  
    - 답변 형식: 답변은 "답변 내용"과 "참고 문서"로 구분해서 작성해주세요. 

CONTEXT : {context}
[INST]QUESTION : {question}[/INST]
"""

prompt = ChatPromptTemplate.from_template(template)
#print(prompt)

# 4. LLM 답변 생성
model = OllamaLLM(model="llama3.2", max_new_tokens= 2048 ,temperature=0.1, base_url='http://180.68.194.221:11434')
chain = prompt | model 

# 5. LLM 응답
result = chain.invoke({"context":content,"question": user_question})
print("\n")
print("LLAMA3.2 답변")
print(result)

백터검색을 참고하여 프롬프트의 컨텐트에 사용할 문서번호를 입력해주세요? :  17


전시 군사법원 운영절차 요약해주세요 에 대한 답변을 LLAMA3.2가 실행중입니다. 잠시후 결과를 확인하세요


In [ ]:
오라클 체인어블 유틸리티를 활용한 문서에서 RAG까지 한번에 처리하는 시나리오를 마칩니다. 감사합니다.